In [4]:
import pandas as pd
import numpy as np
from untrade.client import Client
data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


In [5]:



def process_data(df):
   """
   Process BTC data to generate trading signals based on different strategies.
   Supported strategies: 'bollinger', 'macd', 'moving_average_crossover', 'custom_macd'.


   Parameters:
   df (pd.DataFrame): DataFrame with columns 'open', 'high', 'low', 'close'
   strategy (str): The trading strategy to apply.


   Returns:
   pd.DataFrame: DataFrame with added columns for indicators and trading signals
   """
   # Make a copy of the dataframe
   df = df.copy()


   # Calculate common indicators
   df['RSI'] = calculate_rsi(df['close'], period=14)
   df['ATR'] = calculate_atr(df)
   df['EMA_6'] = df['close'].ewm(span=6, adjust=False).mean()


   # Initialize columns
   df['Signal'] = 0
   df['trade'] = 0
   df['SL'] = np.nan
   df['TP'] = np.nan
   df['trade_type'] = "square-off"


       # MACD Calculation
   df['EMA_12'] = df['close'].ewm(span=5, adjust=False).mean()
   df['EMA_26'] = df['close'].ewm(span=13, adjust=False).mean()
   df['MACD'] = df['EMA_12'] - df['EMA_26']
   df['Signal_Line'] = df['MACD'].ewm(span=5, adjust=False).mean()


   for i in range(1, len(df)):
       # Bullish Signal: MACD Fast crosses above Slow and price is above EMA_6
       if (
           df['MACD'].iloc[i] > df['Signal_Line'].iloc[i]
           and df['MACD'].iloc[i - 1] <= df['Signal_Line'].iloc[i - 1]
           and df['close'].iloc[i] > df['EMA_6'].iloc[i]
       ):
           df.loc[df.index[i], 'Signal'] = 1
           df.loc[df.index[i], 'trade'] = 1
           df.loc[df.index[i], 'trade_type'] = "long"
           df.loc[df.index[i], 'SL'] = df['close'].iloc[i] - 1.5 * df['ATR'].iloc[i]
           df.loc[df.index[i], 'TP'] = df['close'].iloc[i] + 3 * df['ATR'].iloc[i]


       # Bearish Signal: MACD Fast crosses below Slow and price is below EMA_6
       elif (
           df['MACD'].iloc[i] < df['Signal_Line'].iloc[i]
           and df['MACD'].iloc[i - 1] >= df['Signal_Line'].iloc[i - 1]
           and df['close'].iloc[i] < df['EMA_6'].iloc[i]
       ):
           df.loc[df.index[i], 'Signal'] = -1
           df.loc[df.index[i], 'trade'] = -1
           df.loc[df.index[i], 'trade_type'] = "short"
           df.loc[df.index[i], 'SL'] = df['close'].iloc[i] + 1.5 * df['ATR'].iloc[i]
           df.loc[df.index[i], 'TP'] = df['close'].iloc[i] - 3 * df['ATR'].iloc[i]


   return df



def calculate_rsi(series, period=14):
   """Helper function to calculate RSI."""
   delta = series.diff(1)
   gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
   loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
   rs = gain / loss
   return 100 - (100 / (1 + rs))




def calculate_atr(df, period=14):
   """Helper function to calculate ATR."""
   df['TR'] = pd.DataFrame({
       'hl': df['high'] - df['low'],
       'hc': abs(df['high'] - df['close'].shift(1)),
       'lc': abs(df['low'] - df['close'].shift(1))
   }).max(axis=1)
   return df['TR'].rolling(window=period).mean()


def strat(data):
   """
   Apply strategy to filter consecutive signals.


   Parameters:
   - data (DataFrame): Dataframe containing buy/sell signals.


   Returns:
   - data (DataFrame): Dataframe with filtered signals.
   """
   signal = []
   prev = None
   for value in data["Signal"]:
       if value == prev:
           signal.append(0)
       else:
           signal.append(value)
       prev = value


   data["signals"] = signal
   data['leverage'] = 4

   # Keep only the required columns
   data = data[['datetime', 'open', 'high', 'low',
            'close', 'volume', 'signals', 'trade_type', 'leverage']]


   return data


def perform_backtest(csv_file_path):
   """
   Perform backtesting using the untrade SDK.


   Parameters:
   - csv_file_path (str): Path to the CSV file containing historical price data and signals.


   Returns:
   - result (generator): Generator object that yields backtest results.
   """
   # Create an instance of the untrade client
   client = Client()


   # Perform backtest using the provided CSV file path
   result = client.backtest(
       jupyter_id="vraj2811",  # your Jupyter ID
       file_path=csv_file_path,
       leverage=1,  # Adjust leverage as needed
   )


   return result




# Load ETH data
data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)


# Save processed data to CSV file
res = strat(res)
res.to_csv("processed_data.csv", index=False)


# Perform backtest on processed data
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)


# # Print the backtest result
for value in backtest_result:
   print(value)

data: {
  "jupyter_id": "vraj2811",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 437,
      "Leverage Applied": 1.0,
      "Winning Trades": 163,
      "Losing Trades": 274,
      "No. of Long Trades": 243,
      "No. of Short Trades": 194,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 37.299771,
      "Winning Streak": 6,
      "Losing Streak": 12,
      "Gross Profit": 18959.96104,
      "Net Profit": 16337.96104,
      "Average Profit": 37.386639,
      "Maximum Drawdown(%)": 42.478093,
      "Average Drawdown(%)": 5.638885,
      "Largest Win": 3234.901362,
      "Average Win": 263.072536,
      "Largest Loss": -794.302091,
      "Average Loss": -96.87176,
      "Maximum Holding Time": "20 days 0:0:0",
      "Average Holding Time": "2 days 0:25:15",
      "Maximum Adverse Excursion": 68.141593,
      "Avera

In [ ]:
print(data.head())  

   Unnamed: 0             datetime    open    high     low   close    volume
0           0  2019-12-01 00:00:00  151.38  151.38  150.09  150.41  4472.159
1           1  2019-12-01 00:15:00  150.50  150.74  150.16  150.30  3656.530
2           2  2019-12-01 00:30:00  150.30  150.83  150.23  150.48  4620.579
3           3  2019-12-01 00:45:00  150.46  150.61  150.18  150.60  4178.315
4           4  2019-12-01 01:00:00  150.61  150.64  148.60  148.78  5234.745


In [11]:
#finding the difference between the closing of the first candle and low of the third candle after this;

lst = []

high = data['high']
low = data['low']

for a in range(len(high)):
    if a+2 < len(high):
        lst.append(low[a+2] - high[a])
    else:
        lst.append(0)

# #using scipy to generate the peaks and troughs of the data separately
# from scipy.signal import find_peaks, find_peaks_cwt

# peaks, _ = find_peaks(data['high'])
# troughs, _ = find_peaks(-data['low'])


In [14]:
from scipy.signal import argrelextrema
peaks = argrelextrema(data['high'].values, np.greater)
troughs = argrelextrema(data['low'].values, np.less)


def find_return_index(high, low):
    """
    Find when the high value returns to the range defined by (low[a+2] - high[a]).

    Parameters:
    - high (list): List of high prices.
    - low (list): List of low prices.

    Returns:
    - dict: Dictionary mapping index `a` to the first future index `b` where high[b] enters the range.
    """
    lst = []
    return_indices = np.zeros(len(high))

    for a in range(len(high)):
        if a + 2 < len(high):
            diff = low[a + 2] - high[a]
            lst.append(diff)
        else:
            lst.append(0)

    for a in range(len(lst)):
        if lst[a] >= 0:
            target_range = (high[a], low[a+2])  # Difference zone
            for b in range(a + 3, len(high)):
                if target_range[0] <= high[b] <= target_range[1]:  # Check if high[b] enters the range
                    return_indices[b] = 1
                    break
            else:
                return_indices[a] = 0  # If no such index is found, mark as -1
        else:
            # implement the inverse;
            target_range = (high[a+2], low[a])
            for b in range(a + 2, len(high)):
                if target_range[0] <= high[b] <= target_range[1]:  # Check if high[b] enters the range
                    return_indices[b] = -1
                    break
            else:
                return_indices[a] = 0

    return return_indices

Final_testing


In [6]:



def process_data(df):
    """
    Process BTC/ETH data and generate buy/sell signals based on custom range strategy.

    Parameters:
    df (pd.DataFrame): DataFrame with 'open', 'high', 'low', 'close'.

    Returns:
    pd.DataFrame: DataFrame with added columns for trading signals.
    """
    df = df.copy()

    # Initialize signals
    df['Signal'] = 0  # Buy (1), Sell (-1), No signal (0)
    df['trade_type'] = "square-off"

    # Store ranges
    buy_ranges = []  # (high[i], low[i+3])
    sell_ranges = []  # (low[i], high[i+3])

    # Identify range conditions
    for i in range(len(df) - 3):
        if df['high'].iloc[i] < df['low'].iloc[i + 3]:
            buy_ranges.append((df.index[i], df['high'].iloc[i], df['low'].iloc[i + 3]))
        elif df['low'].iloc[i] > df['high'].iloc[i + 3]:
            sell_ranges.append((df.index[i], df['low'].iloc[i], df['high'].iloc[i + 3]))

    # Check re-entry into the range and generate signals
    for i in range(len(df)):
        for start_idx, high_val, low_val in buy_ranges:
            if i > start_idx and low_val <= df['low'].iloc[i] <= high_val:  # Price re-enters range
                if all(df['low'].iloc[j] > low_val for j in range(i + 1, min(i + 5, len(df)))):  
                    # Price moves up without touching low_val
                    df.loc[df.index[i], 'Signal'] = 1
                    df.loc[df.index[i], 'trade_type'] = "long"
                    break  # Stop checking once a signal is found

        for start_idx, low_val, high_val in sell_ranges:
            if i > start_idx and low_val <= df['high'].iloc[i] <= high_val:  # Price re-enters range
                if all(df['high'].iloc[j] < high_val for j in range(i + 1, min(i + 5, len(df)))):
                    # Price moves down without touching high_val
                    df.loc[df.index[i], 'Signal'] = -1
                    df.loc[df.index[i], 'trade_type'] = "short"
                    break  # Stop checking once a signal is found

    return df


def strat(data):
   """
   Apply strategy to filter consecutive signals.


   Parameters:
   - data (DataFrame): Dataframe containing buy/sell signals.


   Returns:
   - data (DataFrame): Dataframe with filtered signals.
   """
   signal = []
   prev = None
   for value in data["Signal"]:
       if value == prev:
           signal.append(0)
       else:
           signal.append(value)
       prev = value


   data["signals"] = signal
   data['leverage'] = 4


   # Keep only the required columns
   data = data[['datetime', 'open', 'high', 'low',
            'close', 'volume', 'signals', 'trade_type', 'leverage']]


   return data


def perform_backtest(csv_file_path):
   """
   Perform backtesting using the untrade SDK.


   Parameters:
   - csv_file_path (str): Path to the CSV file containing historical price data and signals.


   Returns:
   - result (generator): Generator object that yields backtest results.
   """
   # Create an instance of the untrade client
   client = Client()


   # Perform backtest using the provided CSV file path
   result = client.backtest(
       jupyter_id="vraj2811",  # your Jupyter ID
       file_path=csv_file_path,
       leverage=1,  # Adjust leverage as needed
   )


   return result




# Load ETH data
data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)


# Save processed data to CSV file
res = strat(res)
res.to_csv("processed_data.csv", index=False)


# Perform backtest on processed data
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)


# Print the backtest result
for value in backtest_result:
   print(value)

data: {
  "jupyter_id": "vraj2811",
  "result_type": "Main",
  "message": "No trades identified/executed.",
  "result": ""
}




In [ ]:

def process_data(df):
    """
    Process BTC/ETH data and generate buy/sell signals based on custom range strategy.

    Parameters:
    df (pd.DataFrame): DataFrame with 'open', 'high', 'low', 'close'.

    Returns:
    pd.DataFrame: DataFrame with added columns for trading signals.
    """
    df = df.copy()

    # Initialize signals
    df['Signal'] = 0  # Buy (1), Sell (-1), No signal (0)
    df['trade_type'] = "square-off"

    # Store ranges
    buy_ranges = []  # (high[i], low[i+3])
    sell_ranges = []  # (low[i], high[i+3])

    # Identify range conditions
    for i in range(len(df) - 3):
        if df['high'].iloc[i] < df['low'].iloc[i + 3]:
            buy_ranges.append((df.index[i], df['high'].iloc[i], df['low'].iloc[i + 3]))
        elif df['low'].iloc[i] > df['high'].iloc[i + 3]:
            sell_ranges.append((df.index[i], df['low'].iloc[i], df['high'].iloc[i + 3]))

    # Check re-entry into the range and generate signals
    for i in range(len(df)):
        # Buy Signal Logic (Price moves up without touching the lower bound of the range)
        for start_idx, high_val, low_val in buy_ranges:
            if i > start_idx and low_val <= df['low'].iloc[i] <= high_val:  # Price re-enters range
                # Check the next 10 candles to see if none of them closes below the low value of the range
                if all(df['low'].iloc[j] > low_val for j in range(i + 1, min(i + 11, len(df)))):
                    df.loc[df.index[i], 'Signal'] = 1
                    df.loc[df.index[i], 'trade_type'] = "long"
                    break  # Stop checking once a signal is found

        # Sell Signal Logic (Price moves down without touching the upper bound of the range)
        for start_idx, low_val, high_val in sell_ranges:
            if i > start_idx and low_val <= df['high'].iloc[i] <= high_val:  # Price re-enters range
                # Check the next 10 candles to see if none of them closes above the high value of the range
                if all(df['high'].iloc[j] < high_val for j in range(i + 1, min(i + 11, len(df)))):
                    df.loc[df.index[i], 'Signal'] = -1
                    df.loc[df.index[i], 'trade_type'] = "short"
                    break  # Stop checking once a signal is found

    return df

# Load ETH data
data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_15m.csv")


# Process data
res = process_data(data)


# Save processed data to CSV file
res = strat(res)
res.to_csv("processed_data.csv", index=False)


# Perform backtest on processed data
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)


# Print the backtest result
for value in backtest_result:
   print(value)

In [23]:

def detect_strong_pinbar(df, body_ratio=0.2, wick_ratio=2):
    """
    Detect strong Pin Bar candlestick pattern based on the body and wick ratio.
    For a strong Pin Bar, the wick must be at least twice the size of the body.

    Parameters:
    - df (pd.DataFrame): DataFrame containing 'open', 'high', 'low', 'close' columns.
    - body_ratio (float): Ratio of the body size relative to the full candle (default is 0.2).
    - wick_ratio (float): Ratio of the wick size relative to the body size (default is 2).

    Returns:
    - pd.Series: Boolean series indicating the presence of a strong Pin Bar (True if Pin Bar, False otherwise).
    """
    pinbar_signal = []

    for i in range(len(df)):
        open_price = df['open'].iloc[i]
        close_price = df['close'].iloc[i]
        high_price = df['high'].iloc[i]
        low_price = df['low'].iloc[i]

        # Calculate body size and wick size
        body_size = abs(close_price - open_price)
        candle_size = high_price - low_price
        upper_wick = high_price - max(open_price, close_price)
        lower_wick = min(open_price, close_price) - low_price

        # Check if the candle is a strong Pin Bar (either bullish or bearish)
        if body_size <= body_ratio * candle_size and (
            upper_wick >= wick_ratio * body_size or lower_wick >= wick_ratio * body_size):
            pinbar_signal.append(True)
        else:
            pinbar_signal.append(False)

    return pd.Series(pinbar_signal, index=df.index)


#new strat

pin_bar_data = detect_strong_pinbar(data)
data['EMA_89'] = data['close'].ewm(span=89, adjust=False).mean()

#strat is that if the EMA lies in the wick of this pin bar then we go long or short depending on the direction of the pin bar


In [7]:
import pandas as pd
import numpy as np

def detect_strong_pinbar(df, body_ratio=0.2, wick_ratio=2):
    """
    Detect strong Pin Bar candlestick pattern based on the body and wick ratio.
    For a strong Pin Bar, the wick must be at least twice the size of the body.

    Parameters:
    - df (pd.DataFrame): DataFrame containing 'open', 'high', 'low', 'close' columns.
    - body_ratio (float): Ratio of the body size relative to the full candle (default is 0.2).
    - wick_ratio (float): Ratio of the wick size relative to the body size (default is 2).

    Returns:
    - pd.Series: Boolean series indicating the presence of a strong Pin Bar (True if Pin Bar, False otherwise).
    """
    pinbar_signal = []

    for i in range(len(df)):
        open_price = df['open'].iloc[i]
        close_price = df['close'].iloc[i]
        high_price = df['high'].iloc[i]
        low_price = df['low'].iloc[i]

        # Calculate body size and wick size
        body_size = abs(close_price - open_price)
        candle_size = high_price - low_price
        upper_wick = high_price - max(open_price, close_price)
        lower_wick = min(open_price, close_price) - low_price

        # Check if the candle is a strong Pin Bar (either bullish or bearish)
        if body_size <= body_ratio * candle_size and (
            upper_wick >= wick_ratio * body_size or lower_wick >= wick_ratio * body_size):
            pinbar_signal.append(True)
        else:
            pinbar_signal.append(False)

    return pd.Series(pinbar_signal, index=df.index)

def process_data(df):
    """
    Process BTC/ETH data and generate buy/sell signals based on custom range strategy 
    and strong Pin Bar strategy.

    Parameters:
    df (pd.DataFrame): DataFrame with 'open', 'high', 'low', 'close'.

    Returns:
    pd.DataFrame: DataFrame with added columns for trading signals, SL, TP.
    """
    df = df.copy()

    # Initialize signals and trade state
    df['Signal'] = 0  # Buy (1), Sell (-1), No signal (0)
    df['trade_type'] = "square-off"
    df['SL'] = np.nan  # Stop-Loss levels
    df['TP'] = np.nan  # Take-Profit levels
    df['trade'] = 0  # Trade state: 0 = no trade, 1 = long, -1 = short
    df['trailing_stop'] = np.nan  # Trailing stop levels

    # Detect strong Pin Bars
    pinbar_signal = detect_strong_pinbar(df)
    df['PinBar'] = pinbar_signal

    # Calculate EMA_89
    df['EMA_89'] = df['close'].ewm(span=89, adjust=False).mean()

    # Strategy: Check for Buy and Sell conditions based on Pin Bar and EMA_89
    for i in range(1, len(df) - 1):
        if df['PinBar'].iloc[i]:
            open_price = df['open'].iloc[i]
            close_price = df['close'].iloc[i]
            high_price = df['high'].iloc[i]
            low_price = df['low'].iloc[i]
            ema89 = df['EMA_89'].iloc[i]

            # For Buy: EMA_89 should lie in the lower wick and next candle closes higher
            if ema89 >= low_price and ema89 <= open_price and close_price < df['close'].iloc[i + 1]:
                # Square off short position if it's active
                if df['trade'].iloc[i - 1] == -1:  # If the previous trade was short
                    df.loc[df.index[i], 'Signal'] = 2  # Square off short position
                    df.loc[df.index[i], 'trade_type'] = "square-off"
                    df.loc[df.index[i], 'trade'] = 0  # Close the short trade

                df.loc[df.index[i], 'Signal'] = 1  # Buy signal
                df.loc[df.index[i], 'trade_type'] = "long"
                # Set Stop Loss (SL) to the lower wick of the Pin Bar
                df.loc[df.index[i], 'SL'] = low_price
                # Set Take Profit (TP) to 3 times the length of the Pin Bar
                pinbar_length = high_price - low_price
                df.loc[df.index[i], 'TP'] = close_price + (3 * pinbar_length)
                df.loc[df.index[i], 'trade'] = 1  # Open long trade

            # For Sell: EMA_89 should lie in the upper wick and next candle closes lower
            elif ema89 <= high_price and ema89 >= close_price and close_price > df['close'].iloc[i + 1]:
                # Square off long position if it's active
                if df['trade'].iloc[i - 1] == 1:  # If the previous trade was long
                    df.loc[df.index[i], 'Signal'] = -2  # Square off long position
                    df.loc[df.index[i], 'trade_type'] = "square-off"
                    df.loc[df.index[i], 'trade'] = 0  # Close the long trade

                df.loc[df.index[i], 'Signal'] = -1  # Sell signal
                df.loc[df.index[i], 'trade_type'] = "short"
                # Set Stop Loss (SL) to the upper wick of the Pin Bar
                df.loc[df.index[i], 'SL'] = high_price
                # Set Take Profit (TP) to 3 times the length of the Pin Bar
                pinbar_length = high_price - low_price
                df.loc[df.index[i], 'TP'] = close_price - (3 * pinbar_length)
                df.loc[df.index[i], 'trade'] = -1  # Open short trade

    return df



def strat(data):
   """
   Apply strategy to filter consecutive signals.


   Parameters:
   - data (DataFrame): Dataframe containing buy/sell signals.


   Returns:
   - data (DataFrame): Dataframe with filtered signals.
   """
   signal = []
   prev = None
   for value in data["Signal"]:
       if value == prev:
           signal.append(0)
       else:
           signal.append(value)
       prev = value


   data["signals"] = signal
   data['leverage'] = 4


   # Keep only the required columns
   data = data[['datetime', 'open', 'high', 'low',
            'close', 'volume', 'signals', 'trade_type', 'leverage']]


   return data


def perform_backtest(csv_file_path):
   """
   Perform backtesting using the untrade SDK.


   Parameters:
   - csv_file_path (str): Path to the CSV file containing historical price data and signals.


   Returns:
   - result (generator): Generator object that yields backtest results.
   """
   # Create an instance of the untrade client
   client = Client()


   # Perform backtest using the provided CSV file path
   result = client.backtest(
       jupyter_id="vraj2811",  # your Jupyter ID
       file_path=csv_file_path,
       leverage=4,  # Adjust leverage as needed
   )


   return result


# Load ETH data
data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)


# Save processed data to CSV file
res = strat(res)
res.to_csv("processed_data.csv", index=False)


# Perform backtest on processed data
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)


# Print the backtest result
for value in backtest_result:
   print(value)



data: {
  "jupyter_id": "vraj2811",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 23,
      "Leverage Applied": 4.0,
      "Winning Trades": 15,
      "Losing Trades": 8,
      "No. of Long Trades": 7,
      "No. of Short Trades": 16,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 65.217391,
      "Winning Streak": 6,
      "Losing Streak": 2,
      "Gross Profit": 1667.862014,
      "Net Profit": 1529.862014,
      "Average Profit": 66.51574,
      "Maximum Drawdown(%)": 164.029163,
      "Average Drawdown(%)": 47.396812,
      "Largest Win": 3734.235481,
      "Average Win": 694.340898,
      "Largest Loss": -6819.296856,
      "Average Loss": -1110.656433,
      "Maximum Holding Time": "190 days 15:59:59",
      "Average Holding Time": "40 days 0:31:18",
      "Maximum Adverse Excursion": 249.625768,
      "Av